### Script 2.5, Saloni Khandelwal
### Topic Detection: BERT

In [ ]:
#pip install bertopic

In [ ]:
import pandas as pd
import pyarrow
import re

from bertopic import BERTopic
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_parquet('/content/drive/MyDrive/sample_preprocessed_news_final_1.parquet')
df.head()

Mounted at /content/drive


,url,date,language,title,text,cleaned_text,important_words
66957,https://www.deccanherald.com/business/korean-a...,2021-05-24,en,Korean AI tutor Riiid raises $175 million from...,\nKorean AI tutor Riiid raises $175 million fr...,Critics say the pandemic and technology have f...,Critics say pandemic technology widened educat...
106944,https://www.newsbreak.com/news/2754964846047/a...,2022-09-24,en,After Yang review – what to do when your AI ro...,After Yang review – what to do when your AI ro...,After Yang review – what to do when your AI ro...,Yang review AI robot child goes wrong NewsBrea...
97816,https://www.businessinsider.com.au/google-deep...,2021-08-27,en,"Google is pulling the plug on Streams, the cli...","\n\nGoogle is pulling the plug on Streams, the...",The tech giant took control of the app after a...,tech giant took control app absorbing AI firm ...
130453,https://gizmodo.com/mental-health-therapy-app-...,2023-01-09,en,Mental Health App Koko Tested ChatGPT on Its U...,Mental Health App Koko Tested ChatGPT on Its U...,Mental Health App Koko Tested ChatGPT on Its U...,Mental Health App Koko Tested ChatGPT UsersThe...
15751,https://www.khon2.com/technology/ap-technology...,2023-03-31,en,Italy temporarily blocks ChatGPT over privacy ...,\n\n \n\n \n\nItaly temporarily blocks ChatG...,Italy temporarily blocks ChatGPT over privacy ...,Italy temporarily blocks ChatGPT privacy conce...


In [ ]:
%%time
df['important_words'] = df['important_words'].str.lower()
word_list = df['important_words'].tolist()

CPU times: user 9.79 ms, sys: 0 ns, total: 9.79 ms
Wall time: 9.8 ms


In [ ]:
%%time

#min_topic_size=30

modelBert = BERTopic(calculate_probabilities=True, verbose=True, nr_topics=None, top_n_words= 15)
topics, probs = modelBert.fit_transform(word_list)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2023-05-26 17:11:38,134 - BERTopic - Transformed documents to Embeddings
2023-05-26 17:11:50,992 - BERTopic - Reduced dimensionality
2023-05-26 17:11:51,086 - BERTopic - Clustered reduced embeddings


CPU times: user 22.1 s, sys: 1.39 s, total: 23.4 s
Wall time: 36 s


In [ ]:
topic_freq = modelBert.get_topic_freq()
num_topics = len(topic_freq)
print("Number of Topics:", num_topics)

Number of Topics: 21


In [ ]:
freq = modelBert.get_topic_info()

print(f"Topics: {freq.shape[0]}")
freq.head(17)

Topics: 21


,Topic,Count,Name
0,-1,391,-1_ai_data_new_said
1,0,147,0_market_report_global_analysis
2,1,89,1_health_medical_patients_clinical
3,2,41,2_ai_customers_000_customer
4,3,32,3_cloud_data_ai_new
5,4,31,4_chatgpt_ai_posted_media
6,5,30,5_chatgpt_italian_openai_privacy
7,6,23,6_shares_stock_fund_inc
8,7,22,7_ai_cyber_military_us
9,8,22,8_learning_ai_models_climate


In [ ]:
for topic_id in range(num_topics):
    topic_words = modelBert.get_topic(topic_id)
    print(f"Topic {topic_id}: {topic_words}")

Topic 0: [('market', 0.09729505735998412), ('report', 0.060168497825560885), ('global', 0.03884243776567553), ('analysis', 0.036823034314463286), ('research', 0.03215634623620472), ('growth', 0.03067114684951851), ('key', 0.030016334846192032), ('intelligence', 0.029140219526344262), ('artificial', 0.028881461704167184), ('industry', 0.02714071310550093), ('players', 0.02435367277577605), ('data', 0.022374444422301708), ('trends', 0.019823671221792658), ('study', 0.01936701881568285), ('forecast', 0.018635174382880433)]
Topic 1: [('health', 0.018321959574744535), ('medical', 0.01586050811741221), ('patients', 0.0151948355726399), ('clinical', 0.01446692422321312), ('data', 0.013130779859356484), ('pubmed', 0.013058102802456938), ('ai', 0.012919118200248632), ('healthcare', 0.012474281484623121), ('imaging', 0.01158239236739842), ('drug', 0.010957866772337886), ('covid19', 0.010691920953180516), ('using', 0.010515143348895853), ('patient', 0.010081036543390627), ('colonies', 0.009750220

In [ ]:
modelBert.visualize_topics()

In [ ]:
modelBert.get_topic_freq()

,Topic,Count
0,-1,391
1,0,147
2,1,89
3,2,41
4,3,32
5,4,31
6,5,30
7,6,23
9,9,22
10,7,22


In [ ]:
modelBert.visualize_barchart(top_n_topics=17)

In [ ]:
df.shape

(1000, 7)

In [ ]:
df['bert_topic'] = topics
df.shape

(1000, 8)

Topic 0: Market Analysis and Research: This topic is focused on market analysis and research, including reports, global trends, growth, intelligence, and key players in various industries.

Topic 1: Healthcare and Medical AI: This topic revolves around healthcare and medical applications of AI, including topics such as patient care, clinical research, imaging, and drug development.

Topic 2: Chatbots and AI in Technology: This topic explores the use of AI in technology, particularly in the context of chatbots, search engines, and the influence of companies like Microsoft, Google, and OpenAI.

Topic 3: Ethical Considerations of AI: This topic delves into the ethical implications of AI, discussing topics such as advice, human rights, the potential for unethical use of AI systems, cybersecurity, and military applications.

Topic 4: AI and Cloud Computing: This topic focuses on the intersection of AI and cloud computing, including discussions on data processing, machine learning, real-time processing, and the role of companies like Qualcomm and Realme.

Topic 5: AI in Digital Transformation: This topic centers around the impact of AI on digital transformation, customer experiences, data-driven platforms, and the potential for automation and innovation in various industries.

Topic 6: AI in Climate and Environmental Sciences: This topic explores the role of AI in climate and environmental sciences, discussing flood monitoring, climate change, data analysis, and the use of AI for better understanding and decision-making.

Topic 7: Stock Market and Investments: This topic focuses on the stock market, investments, and financial trends, discussing shares, funds, soundhound, institutional investors, and average quarterly performance.

Topic 8: AI in Art and Creativity: This topic revolves around the intersection of AI and art, discussing creative applications, style generation, streetwear, design, and the impact of AI on artistic work and creativity.

Topic 9: AI and Search Engines: This topic explores the use of AI in search engines, discussing companies like Google, Baidu, Bing, and the role of AI in improving search results, language processing, and user experience.

Topic 10: Elon Musk, Tesla, and AI: This topic centers around Elon Musk, Tesla, and their involvement in AI advancements, autonomous driving, and the impact of AI technologies on the automotive industry.

Topic 11: AI in Education and Academic Writing: This topic focuses on the use of AI in education, discussing applications such as generative models, essay writing, plagiarism detection, and the potential impact of AI on student learning and academic integrity.

Topic 12: NVIDIA, GPUs, and AI Computing: This topic explores the role of NVIDIA, GPUs, and AI computing, discussing stock performance, cloud computing, data processing, and the impact of NVIDIA's technology on the AI landscape.

Topic 13: AI in Music and Audio: This topic revolves around the intersection of AI and the music industry, discussing platforms like Spotify, AI-generated music, voice assistants, and the impact of AI on the creation and consumption of music.

Topic 14: AI Policy and International Implications: This topic explores AI policy and its international implications, discussing topics such as AI adoption in India, OECDAI initiatives, national policies, and the impact of AI on different countries.

Topic 15: AI in Education and Skill Development: This topic focuses on AI in education and skill development, discussing programs, university research, data science skills, and the role of AI in shaping the future of education.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df.head()

,url,date,language,title,text,cleaned_text,important_words,bert_topic
66957,https://www.deccanherald.com/business/korean-a...,2021-05-24,en,Korean AI tutor Riiid raises $175 million from...,\nKorean AI tutor Riiid raises $175 million fr...,Critics say the pandemic and technology have f...,critics say pandemic technology widened educat...,-1
106944,https://www.newsbreak.com/news/2754964846047/a...,2022-09-24,en,After Yang review – what to do when your AI ro...,After Yang review – what to do when your AI ro...,After Yang review – what to do when your AI ro...,yang review ai robot child goes wrong newsbrea...,1
97816,https://www.businessinsider.com.au/google-deep...,2021-08-27,en,"Google is pulling the plug on Streams, the cli...","\n\nGoogle is pulling the plug on Streams, the...",The tech giant took control of the app after a...,tech giant took control app absorbing ai firm ...,1
130453,https://gizmodo.com/mental-health-therapy-app-...,2023-01-09,en,Mental Health App Koko Tested ChatGPT on Its U...,Mental Health App Koko Tested ChatGPT on Its U...,Mental Health App Koko Tested ChatGPT on Its U...,mental health app koko tested chatgpt usersthe...,4
15751,https://www.khon2.com/technology/ap-technology...,2023-03-31,en,Italy temporarily blocks ChatGPT over privacy ...,\n\n \n\n \n\nItaly temporarily blocks ChatG...,Italy temporarily blocks ChatGPT over privacy ...,italy temporarily blocks chatgpt privacy conce...,5


In [ ]:
df[df['bert_topic'] == 2]

,url,date,language,title,text,cleaned_text,important_words,bert_topic
42899,https://www.cbs17.com/news/national-news/meet-...,2023-01-15,en,"Meet Tori, AI for your fast food order | CBS 17","\n\n \n\n \n\nMeet Tori, AI for your fast fo...","Meet Tori, AI for your fast food order | CBS 1...",meet tori ai fast food order cbs 17 cbs17com r...,2
167114,https://www.einpresswire.com/article/628575255...,2023-04-18,en,Co-founder of Generative AI Company Persado...,\n Co-founder of Generative AI Company Persad...,Co-founder of Generative AI Company Persado to...,cofounder generative ai company persado speak ...,2
509,https://www.informnny.com/news/tech-news/amazo...,2022-08-18,en,"Amazon testing TikTok-style feed on its app, A...",\n\n \n\nAmazon testing TikTok-style feed on i...,"Amazon testing TikTok-style feed on its app, A...",amazon testing tiktokstyle feed app ai firm sa...,2
102581,https://seekingalpha.com/news/3845275-pinteres...,2022-06-02,en,Pinterest agrees to acquire AI shopping platfo...,Pinterest agrees to acquire AI shopping platfo...,Pinterest agrees to acquire AI shopping platfo...,pinterest agrees acquire ai shopping platform ...,2
89668,https://itbusinessnet.com/2022/07/mindbreeze-i...,2022-07-09,en,Mindbreeze is Recognized in KMWorld’s AI 50: T...,\n\nMindbreeze is Recognized in KMWorld’s AI 5...,"This year’s list, selected by KMWorld, highlig...",years list selected kmworld highlights 50 prom...,2
11988,https://www.wafb.com/prnewswire/2022/11/09/ant...,2022-11-09,en,Antelope Enterprise Subsidiary Kylin Cloud Ent...,Antelope Enterprise Subsidiary Kylin Cloud Ent...,Antelope Enterprise Subsidiary Kylin Cloud Ent...,antelope enterprise subsidiary kylin cloud ent...,2
80090,https://www.goskagit.com/stealth-monitoring-su...,2023-04-13,en,Stealth Monitoring Supercharges Artificial Int...,\nStealth Monitoring Supercharges Artificial I...,Stealth Monitoring Supercharges Artificial Int...,stealth monitoring supercharges artificial int...,2
77646,https://cio.economictimes.indiatimes.com/news/...,2022-03-23,en,ai studio: Icertis releases AI studio to ...,\n\n\n ai studio: Icertis releases AI studi...,Please review and accept these changes below t...,please review accept changes continue using we...,2
190025,https://www.benzinga.com/pressreleases/22/04/n...,2022-08-17,en,Humantic AI Announces $1.5M in First-Round Fun...,Humantic AI Announces $1.5M in First-Round Fun...,"""We are building Humantic AI to help salespeop...",building humantic ai help salespeople understa...,2
76250,https://www.ncnonline.net/accenture-to-acquire...,2023-03-21,en,Accenture to Acquire Industrial AI Company Flu...,\n\n\nAccenture to Acquire Industrial AI Compa...,Flutura will strengthen Accenture’s industrial...,flutura strengthen accentures industrial ai se...,2


In [ ]:
modelBert.get_topic_info()

,Topic,Count,Name
0,-1,391,-1_ai_data_new_said
1,0,147,0_market_report_global_analysis
2,1,89,1_health_medical_patients_clinical
3,2,41,2_ai_customers_000_customer
4,3,32,3_cloud_data_ai_new
5,4,31,4_chatgpt_ai_posted_media
6,5,30,5_chatgpt_italian_openai_privacy
7,6,23,6_shares_stock_fund_inc
8,7,22,7_ai_cyber_military_us
9,8,22,8_learning_ai_models_climate


In [ ]:
#df[df['bert_topic'] == 2]

In [ ]:
topic_2_df = df[df['bert_topic'] == 2]
topic_2_freq = topic_2_df['date'].value_counts().sort_index()
#topic_2_freq

In [ ]:
# topic_2_df = df[df['bert_topic'] == 2]
# topic_2_freq = topic_2_df['date'].value_counts().sort_index()

# plt.figure(figsize=(10, 6))
# topic_2_freq.plot(kind='line')
# plt.xlabel('Publication Date')
# plt.ylabel('Frequency')
# plt.title('Publication Frequency of AI Chatbots Over Time')
# plt.xlim(['2023-01-01', '2023-04-30'])
# plt.grid(True)
# plt.show()

In [ ]:
# topic_9_df = df[df['bert_topic'] == 9]
# topic_9_freq = topic_9_df['date'].value_counts().sort_index()

# plt.figure(figsize=(10, 6))
# topic_9_freq.plot(kind='line')
# plt.xlabel('Publication Date')
# plt.ylabel('Frequency')
# plt.title('Publication Frequency of AI in Search Engines Over Time')
# plt.xlim(['2023-01-01', '2023-04-30'])
# plt.grid(True)
# plt.show()

In [ ]:
# topic_15_df = df[df['bert_topic'] == 15]
# topic_15_freq = topic_15_df['date'].value_counts().sort_index()

# plt.figure(figsize=(10, 6))
# topic_15_freq.plot(kind='line')
# plt.xlabel('Publication Date')
# plt.ylabel('Frequency')
# plt.title('Publication Frequency of AI in Education Over Time')
# plt.xlim(['2021-01-01', '2023-04-30'])
# plt.grid(True)
# plt.show()

In [ ]:
# topic_df = df[df['bert_topic'] == 3]
# topic_freq = topic_df['date'].value_counts().sort_index()

# plt.figure(figsize=(10, 6))
# topic_freq.plot(kind='line')
# plt.xlabel('Publication Date')
# plt.ylabel('Frequency')
# plt.title('Publication Frequency of AI for Human Rights Over Time')
# plt.xlim(['2023-01-01', '2023-04-30'])
# plt.grid(True)
# plt.show()

In [ ]:
# topic_df = df[df['bert_topic'] == 1]
# topic_freq = topic_df['date'].value_counts().sort_index()

# plt.figure(figsize=(10, 6))
# topic_freq.plot(kind='line')
# plt.xlabel('Publication Date')
# plt.ylabel('Frequency')
# plt.title('Publication Frequency of AI in Healthcare Over Time')
# plt.xlim(['2023-01-01', '2023-04-30'])
# plt.grid(True)
# plt.show()

In [ ]:
# topic_df = df[df['bert_topic'] == 8]
# topic_freq = topic_df['date'].value_counts().sort_index()

# plt.figure(figsize=(10, 6))
# topic_freq.plot(kind='line')
# plt.xlabel('Publication Date')
# plt.ylabel('Frequency')
# plt.title('Publication Frequency of AI in Art Over Time')
# plt.xlim(['2023-01-01', '2023-04-30'])
# plt.grid(True)
# plt.show()

In [ ]:
# import pyarrow as pa
# import pyarrow.parquet as pq

In [ ]:
# df.head()

In [ ]:
# table = pa.Table.from_pandas(df)

In [ ]:
# local_path = '/content/drive/MyDrive/BERT_topic_df.parquet'
# pq.write_table(table, local_path)

In [ ]:
import os

os.environ['LC_ALL'] = 'en_US.UTF-8'

In [ ]:
# !jupyter nbconvert --to html /content/BERT_KeyTopics_NLP_Final.ipynb